In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from PIL import Image
import caffe
import setproctitle
import os, sys
import score
import tools
from copy import copy
import time
import setup
import training_tools as tt

In [2]:
setproctitle.setproctitle(os.path.basename(os.getcwd()))

caffe_root = '/home/cv/hdl/caffe'
models = '{}/models'.format(caffe_root)
voc_dir = '{}/data/pascal/VOC/VOC2010'.format(caffe_root)
snapshot = 'snapshot'
part1 = 'head'
part2 = 'torso'
joint_parts = 'head+torso'
parts = [part1, part2, joint_parts]

# generated using **new** fcn_net_surgery.ipynb
base_weights = 'vgg_no_bilinear_vgg16fc.caffemodel'
classes = np.asarray(['background', 'head', 'torso', 'head+torso', 'left arm', 'right arm', 'arms', 'left leg', 'right leg', 'legs', 'person'])

In [3]:
# setup
t = time.time()

In [9]:
# init
caffe.set_device(0)
caffe.set_mode_gpu()

# load net
net_joint = tt.get_net(joint_parts)

layers = [l for l in net_joint.params.keys() if 'score' not in l]

# load solvers
net_1_solver = tt.get_solver(part1)
net_2_solver = tt.get_solver(part2)
net_joint_solver = tt.get_solver(joint_parts)

# load pre-trained nets 1 & 2
net_1_solver.net.copy_from(tt.get_caffemodel(part1, 44000))
net_2_solver.net.copy_from(tt.get_caffemodel(part2, 80000))

# get max params between nets 1 and 2
maximas = tt.max_params(net_1_solver.net, net_2_solver.net, layers)

# save newly initialized joint net
# net_joint.save('part_maximas_net_joint.caffemodel')

# copy max params to joint net
net_joint_params = {l: (net_joint_solver.net.params[l][0].data, net_joint_solver.net.params[l][1].data) for l in layers}
tt.copy_params(maximas, net_joint_params, layers)

# copy max params to joint net
# net_joint_solver.net.copy_from(weights)

# surgeries (for bilinear interpolation in deconv layers)
tt.do_surgery(net_joint_solver)

net_joint_solver.test_nets[0].share_with(net_joint_solver.net)

val_joint = np.loadtxt('{}/ImageSets/person/{}_val.txt'.format(voc_dir, joint_parts), dtype=str)
seg_results = '{}/model/segmentation_results'.format(joint_parts)

In [10]:
for _ in range(800):
    net_joint_solver.step(100)

KeyboardInterrupt: 